In [ ]:
def find_threshold(matrix,cut_percentage):
    # --- 2. 閾値計算のためのデータ抽出 ---
    import matplotlib.ticker as ticker

    # Φ (行列) から、0より大きいエッジの重みだけをすべて取り出す
    # (上三角行列または下三角行列のみを対象にし、重複カウントを避ける)
    indices = np.triu_indices(len(matrix), k=1) # k=1で対角成分を除く
    edge_weights = matrix[indices]

    # 0より大きい（実際に存在する）エッジの重みだけを抽出
    existing_edges = edge_weights[edge_weights > 0]

    if len(existing_edges) == 0:
        print("エラー: グラフにエッジが存在しません。")
    else:
        # --- 3. 閾値の決定 (アプローチ1) ---
        # np.percentile を使い、指定したパーセンタイルの値を計算する
        # これが求める閾値 τ (tau) となる
        # (95パーセンタイルの値 = 下から数えて95%地点の値)
        tau = np.percentile(existing_edges, cut_percentage)

        print(f"--- 閾値計算結果 ---")
        print(f"全エッジ数（0より大）: {len(existing_edges)}")
        print(f"弱いエッジを {cut_percentage}% カットする場合...")
        print(f"決定された閾値 (τ): {tau:.7f}")
        print(f"この閾値 τ より大きい（残すべき）エッジの数: {np.sum(existing_edges > tau)}")
        print(f"残るエッジの割合: {np.sum(existing_edges > tau) / len(existing_edges) * 100:.2f}%")


        # --- 4. 累積確率プロット（CDF）の作成と可視化 ---

        sorted_weights = np.sort(existing_edges)
        # Y軸（累積確率）を作成 (1/N, 2/N, ..., N/N)
        y_cumulative = np.arange(1, len(sorted_weights) + 1) / len(sorted_weights)

        plt.figure(figsize=(12, 7))
        # 累積確率プロット (CDF)
        plt.plot(sorted_weights, y_cumulative, color='blue', label='Cumulative Distribution (CDF)')

        # X軸を対数スケールにすると、べき乗則の「裾」が見やすくなる (オプション)
        # plt.xscale('log')

        # --- 5. 決定した閾値をグラフ上に表示 ---
        # 閾値 τ で垂直線を引く
        plt.axvline(x=tau, color='red', linestyle='--',
                    label=f'Threshold τ = {tau:.7f}')

        # 累積確率 cut_percentage/100 の点で水平線を引く
        plt.axhline(y=cut_percentage / 100.0, color='red', linestyle=':',
                    label=f'Cumulative Probability {cut_percentage / 100.0:.2f}')

        # 交点に印をつける
        plt.plot(tau, cut_percentage / 100.0, 'ro') # 'ro' = red circle marker

        plt.title('Cumulative Distribution Function (CDF) of Edge Weights')
        plt.xlabel('Edge Weight (要素の値)')
        plt.ylabel('Cumulative Probability (累積確率)')
        plt.legend()
        plt.grid(True, which="both", ls="--", alpha=0.5)

        # Y軸をパーセンテージ表示にすると見やすい
        plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

        plt.show()

In [ ]:
def adj_matrix_uni(topic_term_dists_uni,vocab_uni,threshold):
    co_occurrence_uni = np.dot(topic_term_dists_uni.T, topic_term_dists_uni)
    co_matrix_uni = pd.DataFrame(co_occurrence_uni, index=vocab_uni, columns=vocab_uni)
    threshold = np.quantile(co_matrix_uni.values, threshold)
    filtered_uni = co_matrix_uni.mask(co_matrix_uni < threshold, 0)
    adj_matrix_uni = filtered_uni.to_numpy(dtype=float)
    return adj_matrix_uni

In [ ]:
def plot_igraph_from_adjacency_top20(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # 隣接行列をnumpy配列に変換
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # エッジの重みが閾値以下のものは0にする
    adj = np.where(adj > edge_threshold, adj, 0)

    # igraphグラフ作成
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # 孤立ノード除去
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # ノードラベル
    # labels = vocab として渡される前提
    if "name" in g.vs.attributes():
        # g.vs["name"] から g.vs["label"] へコピー
        g.vs["label"] = g.vs["name"]
    else:
        # "name" がない場合のフォールバック
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # 次数の高いノードの上位top_n_labels件のみラベルを付与
    degrees = g.degree()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # 上位top_n_labels件
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    final_labels = []
    for i in range(g.vcount()):
        if i in top_indices:
            final_labels.append(g.vs[i]["label"])
        else:
            final_labels.append("")
    g.vs["label"] = final_labels
    # レイアウト
    layout = g.layout(layout_type)

    # エッジの太さ
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # コミュニティ検出（例: Louvain法, fallback to fastgreedy if Louvain not available）
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # ノードの次数に応じてノードサイズを変更する
    degrees = g.degree()
    # 適度なスケーリングのために最小サイズと最大サイズを指定
    min_size = 20
    max_size = 80
    # スケーリング
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # ノードサイズ算出
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # ノードサイズに基づき再描画（ノードの大きさ＝次数）
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

# 評価指標

In [ ]:
import pandas as pd
import numpy as np
import igraph as ig
from scipy import sparse
def get_node_metrics_fast(graph, partition=None):
    """
    行列演算を用いた高速版: 参加係数とコミュニティ内次数(z-score)を計算
    """
    if partition is None:
        partition = graph.community_multilevel()

    membership = np.array(partition.membership)
    num_communities = len(partition)
    num_nodes = graph.vcount()

    # --- 高速化の前準備: 疎行列の作成 ---
    A = graph.get_adjacency_sparse()

    # U: 所属行列 (Membership Matrix) - ノード x コミュニティ
    # U[i, c] = 1 (ノードiがコミュニティcに属する場合), 0 (それ以外)
    row_indices = np.arange(num_nodes)
    col_indices = membership
    data = np.ones(num_nodes)
    U = sparse.csr_matrix((data, (row_indices, col_indices)), shape=(num_nodes, num_communities))

    # K_dist: コミュニティ別次数行列 (Distribution Matrix)
    # K_dist[i, c] = ノードi が コミュニティc のメンバーとつながっている本数
    # 数式: A (Nodes x Nodes) @ U (Nodes x Comms) -> (Nodes x Comms)
    K_dist = A @ U

    # k_i: 各ノードの全次数 (ベクトル)
    # 疎行列の行ごとの和（np.matrix形式になるのでarrayに変換）
    k_i = np.array(A.sum(axis=1)).flatten()

    # --- A. 参加係数 (Participation Coefficient) の計算 ---
    # P_i = 1 - sum( (k_is / k_i)^2 )

    # 計算用に次数0のノードを1に置換（ゼロ除算回避）。後で結果を0に戻す。
    k_i_safe = np.where(k_i == 0, 1, k_i)

    # K_dist の各要素を2乗
    K_dist_sq = K_dist.power(2)

    # 行ごとに和をとる (sum(k_is^2))
    sum_k_is_sq = np.array(K_dist_sq.sum(axis=1)).flatten()

    # 計算実行
    participation_coeffs = 1.0 - (sum_k_is_sq / (k_i_safe ** 2))

    # 次数0だったノードは参加係数も0にしておく
    participation_coeffs[k_i == 0] = 0.0

    # --- B. コミュニティ内次数 (z-score) の計算 ---
    # z_i = (k_in - mean_k) / std_k

    # k_in: 自分の所属するコミュニティへのリンク数
    # K_dist行列から、自分が所属するコミュニティの列の値だけを抜き出す
    # numpyのファンシーインデックスを使用
    # K_distは疎行列なので、一度toarray()するか、効率的に抜き出す必要がある
    # ※コミュニティ数が数千程度なら toarray() してもメモリは大丈夫だが、
    # 安全のため疎行列のまま対角成分的なものを抜く処理を行う

    # 自分のコミュニティIDに対応する列インデックスを取得
    k_in = np.array(K_dist[np.arange(num_nodes), membership]).flatten()

    # ここからはPandasを使ってグループごとの正規化を一気に行う
    df_temp = pd.DataFrame({
        'comm_id': membership,
        'k_in': k_in
    })

    # コミュニティごとの平均と標準偏差を計算し、各ノードの行にマッピング(transform)
    grouped = df_temp.groupby('comm_id')['k_in']
    means = grouped.transform('mean')
    stds = grouped.transform('std')

    # stdが0（メンバーが1人、または全員同じ次数）の場合は z=0 にする
    stds = stds.replace(0, 1) # ゼロ除算回避

    z_scores = (df_temp['k_in'] - means) / stds
    # 元々stdが0だった場所やNaNになった場所を0に戻す
    z_scores = z_scores.fillna(0).values

    # --- C. データのまとめ ---

    # 必要なら他の指標もここで計算（igraphのメソッドはC言語実装なので高速です）
    degree = k_i # すでに計算済み
    betweenness = graph.betweenness()
    df = pd.DataFrame({
        "name": graph.vs["name"],
        "community_id": membership,
        "degree": degree,
        "z_score": z_scores,
        "participation": participation_coeffs,
        "betweenness": betweenness
    })

    # 上位10件を返す
    top10 = df.sort_values("degree", ascending=False).head(10).reset_index(drop=True)

    return top10

def get_top10_per_community(graph, communities):
    """
    各コミュニティ（サブグラフ）ごとに get_top10_node_metrics() を実行し、
    結果を辞書で返す
    """
    results = {}

    for i, nodes in enumerate(communities+1):
        subgraph = graph.subgraph(nodes)

        # 中心性上位10件を取得
        top10_df = get_node_metrics_fast(subgraph,partition=None)
        top10_df["community"] = i
        results[i] = top10_df
    return results

# 使用例

In [ ]:
g = plot_igraph_from_adjacency_top20(adj_matrix_uni_6,labels=vocab_uni,edge_threshold=0.0001332,top_n_labels=15,vertex_label_size=8,vertex_size=50)
communities = g.community_multilevel()  # Louvain法で検出
community_top10 = get_top10_per_community(g, communities)

for cid, df in community_top10.items():
    print(f"\n=== コミュニティ {cid} ===")
    print(df)